In [17]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, plot_tree

In [11]:
dados = pd.read_csv('dados/dataset_tratado_ml.csv')
dados.head()

,qa_0,qa_1,qa_2,qa_3,qa_4,qa_5,qa_6,qa_7,qa_8,qa_9,qa_10,qa_11,qa_12,qa_13,qa_14,qa_15,qa_16,qa_17,qa_18,qa_19,qa_20,qa_21,qa_22,qa_23,qa_24,qa_25,qa_26,qa_27,qa_28,qa_29,qa_30,qa_31,qa_32,qa_33,qa_34,qa_35,qa_36,qa_37,qa_38,qa_39,qa_40,qa_41,qa_42,qa_43,qa_44,qa_45,qa_46,qa_47,qa_48,qa_49,qa_50,qa_51,qa_52,qa_53,qa_54,qa_55,qa_56,qa_57,qa_58,qa_59,qa_60,qa_61,qa_62,qa_63,qa_64,qa_65,qa_66,qa_67,qa_68,qa_69,qa_70,qa_71,qa_72,qa_73,qa_74,qa_75,qa_76,Formulario,Pontuação Total,cluster
0,-0.222250,1.132626,-0.919340,-0.039814,0.869161,-0.579462,0.027868,-1.502710,1.405424,0.420924,0.580286,-1.072293,0.686686,0.984161,0.883719,-0.259616,0.050236,-0.075785,-0.011180,-0.645653,-0.330144,-0.017097,-1.343778,-1.423855,0.873319,1.025832,-0.441242,1.102301,-1.211921,0.109863,-0.088495,0.659157,0.231585,-0.566756,0.544230,0.398127,0.874539,-0.198302,-0.264779,-0.875606,-0.349890,-1.088745,0.449322,0.628879,0.726660,-0.715887,-0.362612,0.285549,0.228412,0.335453,0.297634,-0.369343,0.565955,-0.066589,-0.770945,-0.141430,0.486493,0.309667,-0.375181,-0.178775,0.512225,0.441271,0.244152,0.428724,-0.681943,0.098967,0.170277,0.030432,0.023072,-0.235182,-0.156095,0.037414,0.116130,-0.251596,0.188349,-0.000564,-0.257168,1,16,0
1,-1.167160,0.146397,-1.900241,1.110827,-0.271690,-1.098713,-0.683391,0.865467,0.939061,-0.239681,0.612857,0.781431,-0.176777,-0.357934,-1.000821,-0.156809,-0.022676,-0.883645,-0.850908,0.470130,-1.073674,0.097795,0.228002,-0.328167,-0.138580,1.074930,-0.573563,-1.215670,0.153438,-0.164659,0.419857,0.319075,-0.425345,0.078396,-0.275985,-0.746371,1.145798,-0.417332,0.644122,-1.273264,-0.355056,0.586626,-0.430281,0.993896,-0.117625,0.808663,0.085127,-0.249815,0.849361,0.081284,-0.214200,0.288030,-0.281877,0.105690,0.003788,0.163188,-0.426162,0.485902,-0.814738,-0.686289,0.514904,-0.240610,-0.132096,-0.079956,-0.294848,-0.227890,-0.782050,-0.303301,0.260375,-0.781844,-0.280805,-0.400740,-0.453500,0.787189,0.299664,-1.009928,0.175925,2,11,0
2,-0.483700,0.681029,-0.491194,2.142845,-0.379312,1.500302,-0.855160,-0.643083,0.287161,-1.205524,-1.212762,-0.467446,-0.803761,-0.421238,1.303725,-0.067832,-0.850501,-0.852181,0.979504,-0.115719,-0.083161,-0.907917,-0.657950,-0.288424,0.242093,0.279091,0.720095,-0.116563,0.895875,-0.770039,1.315857,0.015230,0.077216,0.243875,-0.181459,0.437115,-0.097060,0.526694,-1.113034,0.211710,0.070120,-0.163128,0.193840,-0.392636,0.159361,0.327435,-0.443725,-0.332993,-0.156057,-0.352266,-0.329644,0.460594,-0.760614,-0.532512,-0.255647,0.273827,-0.154544,0.088269,0.193177,-0.080647,0.227297,0.138057,0.272901,-0.057544,-0.058725,0.030112,-0.007731,-0.030894,0.132784,-0.200127,-0.053944,0.060047,0.055370,-0.167659,-0.145972,-0.021702,-0.055030,3,14,0
3,-0.171070,1.087151,-1.083898,1.082242,-0.615085,-1.037482,-0.494665,-1.165432,0.927291,0.046857,1.652730,-1.168870,0.332975,-0.341684,-0.216948,-0.114363,-1.111582,-0.162831,-0.830110,-0.407331,-1.030477,0.473147,-0.128959,-1.736215,0.698360,-0.764919,0.473320,-0.206188,0.645389,0.443801,0.876813,1.038337,0.048821,0.016607,-0.303716,-0.295854,-0.440824,0.495553,0.100992,-0.519707,0.301598,0.006341,-0.189692,0.548221,0.098751,0.012478,-0.077318,-0.215026,0.320206,-0.091161,-0.329997,0.187338,-0.823134,-0.153350,-0.099877,-0.771085,0.036937,0.001141,0.755518,0.459938,-0.344364,-0.078870,-0.837527,-0.816421,0.161129,0.345260,0.153783,0.202957,-0.415559,-0.420979,0.352336,0.069719,-0.026642,-0.051440,0.133735,0.141358,0.234010,4,14,0
4,0.297127,0.477376,1.141624,1.810317,-0.938868,1.161780,-1.361702,-0.438920,-0.468019,-0.924272,-0.220142,-0.220636,-0.657978,0.097654,0.184078,0.624342,-1.074905,-0.721079,-0.802633,-0.243035,0.298073,0.400040,-0.343567,0.063617,-0.178288,0.486322,-0.289934,1.661174,-0.795786,0.151133,-0.146315,1.128810,0.395783,-0.185373,-0.188482,0.774734,0.289530,0.882380,-0.195635,0.235200,0.628282,-0.721512,0.892262,-0.297568,-0.468576,-0.212586,-0.487984,-0.614489,0.257107,-0.408270,-0.607852,-0.490929,-0.326864,-0.103959,0.876301,0.427583,

In [13]:
X = dados.drop(['cluster', 'Formulario', 'Pontuação Total'], axis=1)
y = dados.cluster

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [32]:
tree = DecisionTreeClassifier()
tree.fit(X_train, y_train)

DecisionTreeClassifier()

In [19]:
y_pred = tree.predict(X_test)

In [22]:
from sklearn.metrics import classification_report

report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.97      0.96      0.96      1385
           1       0.95      0.96      0.96       626
           2       0.96      0.96      0.96      1292

    accuracy                           0.96      3303
   macro avg       0.96      0.96      0.96      3303
weighted avg       0.96      0.96      0.96      3303



## Ajustes de hiperparâmetros

In [23]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

grid_search = GridSearchCV(estimator=tree, param_grid=param_grid, cv=5)
grid_search.fit(X_train, y_train)
best_params = grid_search.best_params_

In [24]:
best_params

{'max_depth': 20, 'min_samples_leaf': 4, 'min_samples_split': 10}

In [26]:
tree_best = DecisionTreeClassifier(**best_params, random_state=42)
tree_best.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=20, min_samples_leaf=4, min_samples_split=10,
                       random_state=42)

In [29]:
y_pred = tree_best.predict(X_test)

In [30]:
report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.97      0.96      0.96      1385
           1       0.96      0.96      0.96       626
           2       0.96      0.96      0.96      1292

    accuracy                           0.96      3303
   macro avg       0.96      0.96      0.96      3303
weighted avg       0.96      0.96      0.96      3303



In [35]:
y_train_pred = tree_best.predict(X_train)
train_accuracy = accuracy_score(y_train, y_train_pred)

report = classification_report(y_train, y_train_pred)
print(report)

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5491
           1       0.99      0.99      0.99      2581
           2       0.99      0.99      0.99      5136

    accuracy                           0.99     13208
   macro avg       0.99      0.99      0.99     13208
weighted avg       0.99      0.99      0.99     13208



In [34]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(tree_best, X_train, y_train, cv=5, scoring='accuracy')
print("Acurácia média na validação cruzada:", scores.mean())

Acurácia média na validação cruzada: 0.968125761552597
